### Stratégie pour Utiliser le fichie JSON avec GPT-4o en RAG
	1.	Indexer les questions et réponses dans une base vectorielle (FAISS, ChromaDB).
	2.	Lorsqu’un utilisateur pose une question, récupérer la réponse la plus pertinente.
	3.	Envoyer la réponse trouvée + la question originale à GPT-4o pour générer une réponse contextualisée.

### Préparer la Base de Données Vectorielle

On va transformer les instructions et inputs en vecteurs pour effectuer des recherches de similarité.

In [ ]:
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

with open("C:/Users/redaf/OneDrive/Bureau/Datathonclub/Datathon/data/faq.json", "r", encoding="utf-8") as f:
    data = json.load(f)

texts = [entry["instruction"] + " " + entry["input"] for entry in data]
responses = [entry["output"] for entry in data]

model = SentenceTransformer("all-MiniLM-L6-v2",device='cpu')
embeddings = model.encode(texts)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

faiss.write_index(index, "C:/Users/redaf/OneDrive/Bureau/Datathonclub/Datathon/data/faq.faiss")
with open("C:/Users/redaf/OneDrive/Bureau/Datathonclub/Datathon/data/responses.json", "w", encoding="utf-8") as f:
    json.dump(responses, f)

### Rechercher la Réponse la Plus Pertinente

Quand un utilisateur pose une question, on va chercher la réponse la plus proche dans FAISS.

In [ ]:
def search_faq(query):
    query_vector = model.encode([query])  

    index = faiss.read_index("C:/Users/redaf/OneDrive/Bureau/Datathonclub/Datathon/data/faq.faiss")
    with open("C:/Users/redaf/OneDrive/Bureau/Datathonclub/Datathon/data/responses.json", "r", encoding="utf-8") as f:
        responses = json.load(f)

    _, idx = index.search(np.array(query_vector), k=1)  
    return responses[idx[0][0]]

In [ ]:
import datetime

def check_maintenance():
    last_maintenance_date = datetime.datetime(2024, 1, 31)
    current_date = datetime.datetime.now()
    delta = current_date - last_maintenance_date

    if delta.days > 30:
        return "Rappel : Une maintenance est nécessaire. Il y a plus de 30 jours depuis la dernière maintenance."
    else:
        return "Votre fauteuil n'a pas besoin de maintenance, vous êtes à jour!"
check_maintenance()

### Intégrer GPT-4o avec OpenAI

Une fois qu’on a récupéré la réponse la plus pertinente, on l’envoie à GPT-4o en contexte pour une réponse plus fluide.

In [ ]:
from openai import OpenAI
from rapidfuzz import process

client = OpenAI()

def detect_model(question, models):
    """Détecte le modèle de fauteuil roulant avec tolérance aux fautes de frappe."""
    best_match, score, _ = process.extractOne(question, models, score_cutoff=50)
    
    if score: 
        return best_match
    return None  

def ask_gpt4o(question):

    print(f"Check rapide de maintenance : \n{check_maintenance()}\n")

    retrieved_answer = search_faq(question)

    models = [
        "aviva rx40", "quickie q50 r", "permobil f5", "invacare tdx sp2", "aviva fx40",
        "juvo", "lynx", "m3 corpus", "sango", "storm", "tdx", "aviva rx40 ulm"
    ]

    question_lower = question.lower()

    detected_model = detect_model(question_lower, models)

    if not detected_model:
        return "Je n’ai pas reconnu le modèle. Pouvez-vous préciser le modèle exact du fauteuil roulant ?"

    prompt = f"""On est dans le contexte des fauteuils roulants.
    Si aucun modèle est détecté il faut fournir une réponse générale.
Modèle détecté : {detected_model.upper()}
Il faut fournir une réponse en tenant compte du modèle mentionné et de la base de connaissances :
    
Réponse de la base : {retrieved_answer}

Question : {question}

Réponds avec précision en tenant compte du contexte.
"""
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1000
    )
    return response.choices[0].message.content

question = "Comment changer les roues ?"
print(ask_gpt4o(question))


Check rapide de maintenance : 
Rappel : Une maintenance est nécessaire. Il y a plus de 30 jours depuis la dernière maintenance.

Pour changer les roues d'un fauteuil roulant modèle SANGO, veuillez suivre ces étapes :

1. **Préparation :** Assurez-vous d'avoir un espace de travail dégagé et de rassembler les outils nécessaires, tels qu'une clé adaptée pour les boulons de roue et éventuellement un tournevis.

2. **Démontage de la roue existante :** 
   - Positionnez le fauteuil de manière stable.
   - Repérez l'axe de fixation de la roue. Souvent, il s'agit d'une goupille ou d'un écrou. 
   - Retirez la goupille ou desserrez l'écrou à l'aide de la clé pour libérer la roue. Faites attention aux rondelles ou autres petites pièces qui pourraient être présentes.

3. **Installation de la nouvelle roue :**
   - Insérez la nouvelle roue sur l'axe.
   - Assurez-vous que les roulements sont correctement alignés et qu'ils ne présentent pas de jeu.
   - Fixez la roue en remettant en place la goupil